# BEST AVAILABLE PIXEL COMPOSITE (BAP) *in Google Earth Engine Python API*
based on *Pixel-Based Image Compositing for Large-Area Dense
Time Series Applications and Science (White, 2014)*
https://goo.gl/Fi8fCY

## Install Packages
If running the code in a docker instance or google's colaboratory

In [1]:
# import sys
# !{sys.executable} -m pip install geebap
# to upgrade the package run
# !{sys.executable} -m pip install --upgrade geebap

## Make imports

In [2]:
# Import Earth Engine and initialize
import ee
ee.Initialize()

# Import packages
import geebap
from geebap import bap, season, filters, masks, \
                   scores, satcol, functions
from geetools import tools, ui
import pprint

print('version', geebap.__version__)

version 0.1.8dev


## Season
This object holds information of the growing season (start, end and doy *best day of year*). You can make your own, or use 2 pre-made: `Growing_South` and `Growing_North`. This object does not hold any year, just day and month. For example, the pre-made `Growing_South` starts on November 15 (11-15) and ends on March 15 (03-15). But it has a method to add a year, see the example in the code:

In [3]:
a_season = season.Season.Growing_South()
ini, end = a_season.add_year(2000)

print(ini, end)

1999-11-15 2000-03-15


Note that when the season covers two years, start date is in the previous year.

## Collections
The main method to create a BAP uses a `ColGroup` object, that is basically a group of `Collection` objects. You can see all groups:

In [4]:
satcol.ColGroup.options()

{'All': ['LANDSAT/LM1_L1T',
         'LANDSAT/LM2_L1T',
         'LANDSAT/LM3_L1T',
         'LANDSAT/LT04/C01/T1_TOA',
         'LANDSAT/LT04/C01/T1_SR',
         'LANDSAT/LT05/C01/T1_TOA',
         'LANDSAT/LT05/C01/T1_SR',
         'LANDSAT/LE07/C01/T1_TOA',
         'LANDSAT/LE07/C01/T1_SR',
         'LANDSAT/LC08/C01/T1_TOA',
         'LANDSAT/LC08/C01/T1_SR',
         'COPERNICUS/S2',
         'MODIS/006/MYD09GA',
         'MODIS/006/MOD09GA'],
 'Landsat': ['LANDSAT/LM1_L1T',
             'LANDSAT/LM2_L1T',
             'LANDSAT/LM3_L1T',
             'LANDSAT/LT04/C01/T1_TOA',
             'LANDSAT/LT04/C01/T1_SR',
             'LANDSAT/LT05/C01/T1_TOA',
             'LANDSAT/LT05/C01/T1_SR',
             'LANDSAT/LE07/C01/T1_TOA',
             'LANDSAT/LE07/C01/T1_SR',
             'LANDSAT/LC08/C01/T1_TOA',
             'LANDSAT/LC08/C01/T1_SR'],
 'Landsat_Sentinel': ['LANDSAT/LM1_L1T',
                      'LANDSAT/LM2_L1T',
                      'LANDSAT/LM3_L1T',
         

You could also make your own `ColGroup`, but you have to keep in mind that it is composed by `satcol.Collection` objects.
As the process is made to combine pixels from all collections, `Collection` object renames the bands of each collection to match across all, resulting in the following names: BLUE, GREEN, RED, NIR, SWIR, SWIR2, ATM_OP. Also, each collection has methods ready to map a vegetation index: `ndvi`, `evi` and `nbr`. 

## Masks
There is *(by now)* one mask you can include in the process: clouds

In [5]:
cld_mask = masks.Clouds()

## Filters
There are *(by now)* two filters you can use in the process:

**cloud percentage**: `filters.CloudsPercent`

**masked pixel percentage**: `filters.MaskPercent`. This filter can be used **only** if maskpercent score is included in the process.

In [6]:
filt_cld = filters.CloudsPercent()
filt_mask = filters.MaskPercent()

## Scores
This is what makes the difference. Every score has its own parameters, but all share two main params:

**range_out**: the range of values the score will be, by default it is (0, 1)

**sleep**: as the creaton of a BAP composite is a 'concatenated' process, it can make more requests that are allowed, so if you set this parameter, the process will wait those seconds until the next computing.

## White's scores

### DOY (best day of the year)
Basically, pixels from images closer to that date will have higher score
It takes 2 params:

**season**: You can use the same as the one for the process, or not. Each `Season` object has its own *doy*. By default it is `Season.Growing_South`

**formula**: distribution ecuation. There are two (by now) options: `Normal` (https://en.wikipedia.org/wiki/Normal_distribution) or `Exponential` (https://en.wikipedia.org/wiki/Exponential_distribution). Default is `Normal`

In [7]:
doy = scores.Doy()

### Satellite
It uses a list of available satellite for each year that you can check:

In [8]:
# List of satellites for 2000
season.SeasonPriority.relation[2000]

['LANDSAT/LE07/C01/T1_SR',
 'LANDSAT/LE07/C01/T1_TOA',
 'LANDSAT/LT05/C01/T1_SR',
 'LANDSAT/LT05/C01/T1_TOA']

the score has one main param:

**rate**: how much score will decrease for each satellite. For example, for 2000, if rate is 0.05 (default value):

* 'LANDSAT/LE07/C01/T1_SR' --> 1
* 'LANDSAT/LE7_L1T_TOA_FMASK' --> 0.95
* 'LANDSAT/LT05/C01/T1_SR' --> 0.90
* 'LANDSAT/LT5_L1T_TOA_FMASK' --> 0.85

*NOTE: may be the correct name would be 'ratio', so I may change it in the future*

In [9]:
sat = scores.Satellite()

### Atmospheric Opacity
It uses the atmospheric opacity band computed by Surface Reflectance, so only SR collections will have this score. If the process uses a non-SR collection, like TOA, the this score will be zero.

In [10]:
atm_op = scores.AtmosOpacity()

## Distance to mask
This assigns a score regarding the distance of the pixel to the closest masked pixel. As the only mask is for clouds, it could be considered 'distance to cloud'. It has 3 main params:

**unit**: unit to measure distance. Defaults to 'meters'

**dmax**: max distance. Pixel further than this distance will have score 1. Defaults to 600

**dmin**: min distance. Defaults to 0 (next pixel from the maks will have score 0).

In [11]:
dist = scores.CloudDist()

## Custom Scores *(not White's)*

### Outliers
It computes an statistics over the whole collection (in the season) and assigns score regarding the *distance* of each pixel value to that statistic. It has 3 main parameters:

**bands**: a list of bands to include in the process. The process splits the score in the number of given bands. For example, if 4 bands are given, the max score for each band will be 0.25

**process**: one of 'mean' or 'median'. Defaults to 'mean'

**dist**: distance from 'mean' or 'median'. Defaults to 0.7

*NOTE: bands must be in the image, so if you use a vegetation index, be sure to include it in the process*

In [12]:
out = scores.Outliers(("ndvi",))

### Mask percentage
It computes the precentage of masked pixels in the image (not the whole scene). It has one main parameter:

**band**: the band that holds the mask.

In [13]:
maskper = scores.MaskPercent()

### Vegetation Index
This scores is based on the absolute value of the given index, parametrized to `range_out`. The only parameter is **index**: the name of it (*ndvi*, *evi* or *nbr*). Defaults to *ndvi*.

In [14]:
ind = scores.Index("ndvi")

### Multiple years (seasons)
If you want to use images from a range of seasons, and not just only one, this scores prioritizes the main season. Take in count that a season may hold 2 years, but the main is the least (see `Season`). It has 3 main params:

**main_year**: this is the central year. Images from this year (season) will have score 1

**season**: a `Season` object.

**ratio**: amount of score that will decrease as it goes further to the main year. Defaults to 0.05. It is similar to *rate* parameter in `Satellite`.



In [15]:
# Will not use it in the test
multi = scores.MultiYear(2010, a_season, range_out=(0.8, 1))

## Making the composite (BAP)
Next step is to create a `Bap` object. It has the following parameters:

**year**: The main year. Remember that a season can have 2 years.

**range**: for multiyear composites you can specify a 'range' as a tuple: (years before, years after). Defaults to (0, 0).
For example, if `range=(1, 1)` and `year=2000`, will compute 1999, 2000 and 2001.

**colgroup**: `ColGroup` object. If `None` it will use the list computed by `SeasonPriority`. Defaults to `None`.

**season**: a `Season` object

**scores**: a list of `Score` objects

**masks**: a list of `Mask` objects

**filters**: a list of `Filter` objects

**fmap**: custom function to apply before computing scores. At this point bands have new names (NIR, SWIR, etc) and vegetation index is computed.

In [16]:
bap_obj = bap.Bap(year=2010, range=(1, 1),
              season=a_season,
              masks=(cld_mask,),
              scores=(
                  #doy, 
                  #sat, 
                  # atm_op, 
                  # dist, 
                  #out, 
                  #maskper, 
                  #ind,
                  multi,
              ),
              filters=(
                  filt_cld, 
                  # filt_mask
              )
                  
          )

## Define a site

In [17]:
site = ee.Geometry.Polygon([[-71,-42],
                            [-71,-43],
                            [-72,-43],
                            [-72,-42]])

## Finally, compute the composite
`Bap` object has a method named `bestpixel` that creates one image out of the pixels with higher score in the collection (which includes all collections given). It has the following parameters:

**site**: The site where the composite will be computed

**indices**: a list of vegetation indices. Defaults to `None`

**name**: name for the band that holds the final score. Defaults to *score*

**bands**: a list of bands that will be on the image (in case you don't need all). Defaults to `None` which means *include all bands*

**normalize**: normalize the final score to be between 0 and 1. Defaults to `True`

**bbox**: distance of the buffer around the site. Defaults to 0.

**force**: if there are no images for the specified parameters, and this is set to `True`, an empty image will be created. Defaults to `True`

This method return a `namedtuple`, so `.image` will be the composite and `.col` will be the collection

In [20]:
bap_obj.debug = False

In [21]:
composite = bap_obj.fast_composite(site=site, indices=("ndvi",))

Waiting times:
score-multi 0

['LANDSAT/LE07/C01/T1_TOA', 'LANDSAT/LE07/C01/T1_SR', 'LANDSAT/LT05/C01/T1_TOA', 'LANDSAT/LT05/C01/T1_SR']
ImageCollection: LANDSAT/LE07/C01/T1_TOA
year: 2009
year: 2010
year: 2011
ImageCollection: LANDSAT/LE07/C01/T1_SR
year: 2009
year: 2010
year: 2011
ImageCollection: LANDSAT/LT05/C01/T1_TOA
year: 2009
year: 2010
year: 2011
ImageCollection: LANDSAT/LT05/C01/T1_SR
year: 2009
year: 2010
year: 2011


### Get the composite from the results

In [22]:
image = composite.image

## Watch the resulting composite
*it may take a while..*

In [23]:
Map = ui.Map()

In [24]:
Map.show()

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

In [25]:
vis = dict(bands=['NIR', 'SWIR', 'RED'], min=0, max=0.5)

In [26]:
Map.addLayer(image, vis, 'BAP composite')

In [47]:
Map.addLayer(composite.collection.first(), vis, 'First')

In [27]:
Map.centerObject(site)

In [25]:
geebap.ipytools.info2map(Map)

In [26]:
Map.addLayer(image.select('score-multi'), dict(min=0, max=1), 'score multiyear')

In [27]:
ui.eprint(composite.collection.size())

224



In [48]:
chart = ui.chart.Image.series(composite.collection, ee.Geometry.Point([-71.79, -42.75]), properties=['score-multi'], scale=30)

In [49]:
chart.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [40]:
col = composite.collection

In [42]:
ui.eprint(col.first(), notebook=True)

In [43]:
test = tools.imagecollection.linear_function_property(col, 'distance_f04f1f69-569d-4508-85a2-09ead2a8c2a6', mean=0, name='test')

In [45]:
ui.eprint(test.first(), notebook=True)

In [44]:
chart2 = ui.chart.Image.series(test, ee.Geometry.Point([-71.79, -42.75]), properties=['test'], scale=30)

EEException: Collection.iterate: Error in map(ID=21):
Number.subtract: Parameter 'left' is required.

In [ ]:
chart2.render_widget(width=800)